In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import math

# ==========================================
# ЧАСТЬ 1: ДАННЫЕ (ОЦИФРОВАНЫ С ВАШИХ СКРИНШОТОВ)
# ==========================================

# Данные из Таблицы А.6 (Показатели деятельности 40 предприятий)
# Нам нужны только столбцы 1 и 13 для Варианта 4, но я добавлю структуру, чтобы было понятно.
# Значения переписаны со скриншотов 222, 223, 224.

data_task5_values = {
    # "Продуктивність праці" (Показатель 1)
    1: [
        9.26, 9.38, 12.11, 10.81, 9.35, 9.87, 8.17, 9.12, 5.88, 6.3,
        6.22, 5.49, 6.5, 6.61, 4.32, 7.37, 7.02, 8.25, 8.15, 8.72,
        7.65, 8.77, 7.0, 11.06, 9.02, 13.28, 9.27, 6.7, 6.69, 9.42,
        7.24, 5.39, 5.61, 5.59, 6.57, 6.54, 4.23, 5.22, 18.0, 11.03
    ],
    # "Середньорічний фонд зарплати" (Показатель 13)
    13: [
        47750, 50391, 43149, 41089, 14257, 22061, 52509, 14903, 11152, 16821,
        19459, 12973, 50907, 6920, 5736, 26705, 20068, 11487, 32029, 18946,
        19074, 18452, 12096, 7888, 7566, 94697, 29626, 11688, 21955, 12243,
        48141, 20122, 7612, 27404, 39648, 43799, 6235, 11524, 17309, 22225
    ]
}
df_task5 = pd.DataFrame(data_task5_values)

# Данные из Таблицы Б.4 (Оценки экспертов для 13 компетенций)
# Строки - компетенции, Столбцы - Эксперты 1-8.
data_task6_values = {
    'Comp': [
        'Орієнтація на результат', 'Організованість', 'Впливовість', 'Комунікабельність',
        'Впевненість у собі', 'Керування конфліктами', 'Адаптовність', 'Командна робота',
        'Орієнтація на клієнта', 'Активне слухання', 'Відкритість', 'Керування ризиками', 'Гнучкість'
    ],
    1: [5, 5, 5, 3, 5, 4, 4, 5, 5, 4, 5, 5, 4],
    2: [5, 5, 5, 5, 4, 5, 5, 3, 4, 4, 4, 4, 3],
    3: [5, 5, 5, 5, 4, 5, 4, 5, 3, 5, 4, 4, 5],
    4: [5, 5, 4, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4],
    5: [4, 5, 5, 2, 5, 4, 3, 5, 5, 3, 5, 4, 5],
    6: [5, 5, 4, 4, 5, 5, 5, 3, 4, 5, 4, 4, 3],
    7: [5, 5, 3, 5, 4, 5, 4, 5, 4, 2, 4, 5, 5],
    8: [5, 3, 4, 5, 4, 5, 4, 5, 5, 4, 4, 3, 4]
}
df_all_experts = pd.DataFrame(data_task6_values).set_index('Comp')


# ==========================================
# ЧАСТЬ 2: РЕАЛИЗАЦИЯ АЛГОРИТМОВ (ПО МЕТОДИЧКЕ)
# ==========================================

def get_ranks(data):
    """Присвоение рангов (средние для связок)"""
    return data.rank(method='average')

def calculate_ties_correction(ranks):
    """Поправка T на связанные ранги: T = 1/12 * sum(t^3 - t)"""
    values, counts = np.unique(ranks, return_counts=True)
    T = 0
    for t in counts:
        if t > 1:
            T += (t**3 - t) / 12
    return T

# --- ЗАДАНИЕ 5: СПИРМЕН ---
def spearman_correlation_manual(x, y, alpha=0.05):
    n = len(x)
    rx = get_ranks(x)
    ry = get_ranks(y)

    d_sq_sum = np.sum((rx - ry)**2)
    Tx = calculate_ties_correction(rx)
    Ty = calculate_ties_correction(ry)

    # Формула (2.2) для связанных рангов
    num = (1/6)*(n**3 - n) - d_sq_sum - Tx - Ty
    den_x = (1/6)*(n**3 - n) - 2*Tx
    den_y = (1/6)*(n**3 - n) - 2*Ty

    if den_x * den_y == 0:
        rc = 0
    else:
        rc = num / np.sqrt(den_x * den_y)

    # t-критерий Стьюдента
    if abs(rc) >= 1:
        t_stat = np.inf
    else:
        t_stat = abs(rc) * np.sqrt((n - 2) / (1 - rc**2))

    df = n - 2
    t_crit = stats.t.ppf(1 - alpha/2, df)

    return rc, t_stat, t_crit, t_stat > t_crit

# --- ЗАДАНИЕ 5: КЕНДАЛЛ ---
def kendall_correlation_manual(x, y, alpha=0.05):
    n = len(x)
    df_temp = pd.DataFrame({'x': x, 'y': y}).sort_values(by='x')
    y_sorted = df_temp['y'].values

    # S = P - Q
    S = 0
    for i in range(n):
        for j in range(i + 1, n):
            if y_sorted[j] > y_sorted[i]: S += 1
            elif y_sorted[j] < y_sorted[i]: S -= 1

    # Поправки (через V или T)
    rx = get_ranks(df_temp['x']).values
    ry = get_ranks(pd.Series(y_sorted)).values

    def calc_V(ranks):
        values, counts = np.unique(ranks, return_counts=True)
        V = 0
        for t in counts:
            if t > 1: V += (t * (t - 1)) / 2
        return V

    Vx = calc_V(rx)
    Vy = calc_V(ry)

    denom = np.sqrt((0.5*n*(n-1) - Vx) * (0.5*n*(n-1) - Vy))
    rk = S / denom if denom != 0 else 0

    # Z-тест (формула 2.7)
    z_stat = rk / np.sqrt((2 * (2*n + 5)) / (9 * n * (n - 1)))
    u_crit = stats.norm.ppf(1 - alpha/2)

    return rk, abs(z_stat), u_crit, abs(z_stat) > u_crit

# --- ЗАДАНИЕ 6: КОНКОРДАЦИЯ ---
def concordance_coefficient(expert_matrix, alpha=0.05):
    n = expert_matrix.shape[0] # объекты (13)
    m = expert_matrix.shape[1] # эксперты (3)

    ranked_matrix = expert_matrix.apply(get_ranks, axis=0)
    Ri = ranked_matrix.sum(axis=1)
    R_mean = Ri.mean()
    S = np.sum((Ri - R_mean)**2)

    sum_Ti = 0
    for col in ranked_matrix.columns:
        sum_Ti += calculate_ties_correction(ranked_matrix[col])

    numerator = S
    denominator = (1/12)*(m**2)*(n**3 - n) - m * sum_Ti
    W = numerator / denominator if denominator != 0 else 0

    # Хи-квадрат (формула 2.11)
    chi2_calc = m * (n - 1) * W
    df_chi = n - 1
    chi2_crit = stats.chi2.ppf(1 - alpha, df_chi)

    return W, chi2_calc, chi2_crit, chi2_calc > chi2_crit


# ==========================================
# ЧАСТЬ 3: РЕШЕНИЕ ДЛЯ ВАРИАНТА 4
# ==========================================

print("=== РЕЗУЛЬТАТЫ ДЛЯ ВАРИАНТА 4 ===\n")

# --- ЗАДАНИЕ 5 ---
print(">>> ЗАДАНИЕ 5: Корреляция признаков")
# Для варианта 4: Признаки 1 и 13 (см. Табл. Б.3)
X_var = df_task5[1]
Y_var = df_task5[13]

print(f"Анализ признаков: 1 (Продуктивність) и 13 (Фонд зарплати)")
print(f"Количество наблюдений n = {len(X_var)}")

# Спирмен
rc, t_calc, t_table, sig_s = spearman_correlation_manual(X_var, Y_var)
print(f"\n1. Коэффициент Спирмена (rc): {rc:.4f}")
print(f"   Проверка: t_расч = {t_calc:.4f} {' > ' if sig_s else ' <= '} t_крит = {t_table:.4f}")
print(f"   Вывод: {'Зв’язок статистично значущий' if sig_s else 'Зв’язок не значущий'}")

# Кендалл
rk, z_calc, u_table, sig_k = kendall_correlation_manual(X_var, Y_var)
print(f"\n2. Коэффициент Кендалла (rk): {rk:.4f}")
print(f"   Проверка: z_расч = {z_calc:.4f} {' > ' if sig_k else ' <= '} u_крит = {u_table:.4f}")
print(f"   Вывод: {'Зв’язок статистично значущий' if sig_k else 'Зв’язок не значущий'}")

# Вывод о линейности
print("\n3. Характер связи:")
if abs(rk) > abs(rc):
    print("   |rk| > |rc| -> Зависимость скорее нелинейная")
else:
    print("   |rc| >= |rk| -> Зависимость ближе к линейной")


# --- ЗАДАНИЕ 6 ---
print("\n" + "="*40 + "\n")
print(">>> ЗАДАНИЕ 6: Узгодженість експертів")
# Для варианта 4: Эксперты 1, 3, 7 (см. Табл. Б.5)
expert_cols = [1, 3, 7]
print(f"Выбранные эксперты: {expert_cols}")

df_experts_var4 = df_all_experts[expert_cols]

W_val, chi2_calc, chi2_table, sig_w = concordance_coefficient(df_experts_var4)

print(f"\nКоэффициент конкордации (W): {W_val:.4f}")
print(f"Проверка значимости (Хи-квадрат):")
print(f"   Chi2_расч = {chi2_calc:.4f}")
print(f"   Chi2_крит = {chi2_table:.4f}")
print(f"Вывод: {'Думки експертів узгоджені (W значущий)' if sig_w else 'Узгодженість думок низька (W не значущий)'}")

=== РЕЗУЛЬТАТЫ ДЛЯ ВАРИАНТА 4 ===

>>> ЗАДАНИЕ 5: Корреляция признаков
Анализ признаков: 1 (Продуктивність) и 13 (Фонд зарплати)
Количество наблюдений n = 40

1. Коэффициент Спирмена (rc): 0.3274
   Проверка: t_расч = 2.1359  >  t_крит = 2.0244
   Вывод: Зв’язок статистично значущий

2. Коэффициент Кендалла (rk): 0.2231
   Проверка: z_расч = 2.0273  >  u_крит = 1.9600
   Вывод: Зв’язок статистично значущий

3. Характер связи:
   |rc| >= |rk| -> Зависимость ближе к линейной


>>> ЗАДАНИЕ 6: Узгодженість експертів
Выбранные эксперты: [2, 3, 7]

Коэффициент конкордации (W): 0.4612
Проверка значимости (Хи-квадрат):
   Chi2_расч = 16.6027
   Chi2_крит = 21.0261
Вывод: Узгодженість думок низька (W не значущий)


In [4]:
import numpy as np
from scipy.stats import t
import pandas as pd  # Удобно для вывода таблиц

# 1. Исходные данные (из колонок 3 и 14)
X = np.array([9.26, 9.38, 12.11, 10.81, 9.35, 9.87, 8.17, 9.12, 5.88, 6.3,
        6.22, 5.49, 6.5, 6.61, 4.32, 7.37, 7.02, 8.25, 8.15, 8.72,
        7.65, 8.77, 7.0, 11.06, 9.02, 13.28, 9.27, 6.7, 6.69, 9.42,
        7.24, 5.39, 5.61, 5.59, 6.57, 6.54, 4.23, 5.22, 18.0, 11.03])

Y = np.array([47750, 50391, 43149, 41089, 14257, 22061, 52509, 14903, 11152, 16821,
        19459, 12973, 50907, 6920, 5736, 26705, 20068, 11487, 32029, 18946,
        19074, 18452, 12096, 7888, 7566, 94697, 29626, 11688, 21955, 12243,
        48141, 20122, 7612, 27404, 39648, 43799, 6235, 11524, 17309, 22225])

N = len(X)
alpha = 0.05

print(f"=== 1. Исходные данные и параметры ===")
print(f"Объем выборки (n): {N}")
print(f"Уровень значимости (alpha): {alpha}\n")


# 2. Функция для ранжирования (с учетом связанных рангов)
def rank_data(data):
    # Создаем Series для удобства обработки связанных рангов
    s = pd.Series(data)
    # method='average' гарантирует, что связанным значениям присваивается средний ранг
    ranks = s.rank(method='average', ascending=True)
    return ranks.to_numpy()


# 3. Ранжирование данных
R_X = rank_data(X)
R_Y = rank_data(Y)

print("-" * 20)
print(f"=== 2. Ранжирование данных ===")
print(f"Пример рангов X (первые 5): {R_X[:5]}")
print(f"Пример рангов Y (первые 5): {R_Y[:5]}\n")

# 4. Расчет разности рангов (D) и квадрата разности (D^2)
D = R_X - R_Y
D_sq = D ** 2
sum_D_sq = np.sum(D_sq)

# Создание DataFrame для промежуточного вывода
df_results = pd.DataFrame({
    'X': X, 'Y': Y,
    'Rank(X)': R_X, 'Rank(Y)': R_Y,
    'D': D, 'D^2': D_sq
})

print(f"=== 3. Расчет Sum(D^2) ===")
# Выводим первые и последние строки для примера (to_string вместо to_markdown для работы без tabulate)
print(df_results[['X', 'Y', 'Rank(X)', 'Rank(Y)', 'D', 'D^2']].head(5).to_string(index=False))
print("...")
print(df_results[['X', 'Y', 'Rank(X)', 'Rank(Y)', 'D', 'D^2']].tail(5).to_string(index=False, header=False))

print(f"\nСумма квадратов разностей рангов (Sum D^2): {sum_D_sq:.2f}")

# 5. Расчет коэффициента корреляции Спирмена (r_s)
# Формула Спирмена
rs = 1 - (6 * sum_D_sq) / (N * (N ** 2 - 1))

print("-" * 20)
print(f"=== 4. Расчет коэффициента Спирмена (r_s) ===")
print(f"Формула: r_s = 1 - (6 * Sum(D^2)) / (n * (n^2 - 1))")
print(f"Расчет: r_s = 1 - (6 * {sum_D_sq:.2f}) / (40 * (40^2 - 1)) = {rs:.4f}")

print(f"\nКоэффициент корреляции Спирмена (r_s): {rs:.4f}")
# Значение r_s, полученное через scipy.stats.spearmanr: -0.11903 (для проверки)

# 6. Расчет t-статистики (t_st)
# Формула t-статистики: t_st = r_s * sqrt((n - 2) / (1 - r_s^2))
t_stat = abs(rs) * np.sqrt((N - 2) / (1 - rs ** 2))

print("-" * 20)
print(f"=== 5. Проверка значимости (t-критерий) ===")
print(f"Степени свободы (df = n - 2): {N - 2}")
print(f"Формула t-статистики: t_st = r_s * sqrt((n - 2) / (1 - r_s^2))")
print(f"Расчет t_st: {rs:.4f} * sqrt(({N} - 2) / (1 - {rs ** 2:.4f})) = {t_stat:.4f}")

# 7. Определение критического значения (t_kr)
# t_kr = t(alpha/2; n-2) для двустороннего теста
df = N - 2
prob_for_critical_t = 1 - alpha / 2  # Для t-распределения
t_critical = t.ppf(prob_for_critical_t, df)

print(f"\nКритическое значение t_kr:")
print(f"Используем t({alpha}/2; {df}): t(0.025; 38)")
print(f"t_kr = {t_critical:.4f}")

# 8. Сравнение и вывод
print("-" * 20)
print(f"=== 6. Вывод ===")
print(f"Рассчитанная t-статистика (|t_st|): |{t_stat:.4f}| = {abs(t_stat):.4f}")
print(f"Критическое значение (t_kr): {t_critical:.4f}\n")

# Проверка условия |t_st| > t_kr
if abs(t_stat) > t_critical:
    print(f"Результат: Так как |t_st| ({abs(t_stat):.4f}) > t_kr ({t_critical:.4f}), нулевая гипотеза ОТВЕРГАЕТСЯ.")
    print(f"Следовательно, с вероятностью 95% коэффициент корреляции значимо отличается от нуля.")
else:
    print(f"Результат: Так как |t_st| ({abs(t_stat):.4f}) <= t_kr ({t_critical:.4f}), нулевая гипотеза НЕ ОТВЕРГАЕТСЯ.")
    print(f"Следовательно, коэффициент корреляции r_s={rs:.4f} НЕ ЯВЛЯЕТСЯ значимым на уровне alpha=0.05.")

=== 1. Исходные данные и параметры ===
Объем выборки (n): 40
Уровень значимости (alpha): 0.05

--------------------
=== 2. Ранжирование данных ===
Пример рангов X (первые 5): [29. 32. 38. 35. 31.]
Пример рангов Y (первые 5): [35. 37. 33. 32. 14.]

=== 3. Расчет Sum(D^2) ===
    X     Y  Rank(X)  Rank(Y)    D   D^2
 9.26 47750     29.0     35.0 -6.0  36.0
 9.38 50391     32.0     37.0 -5.0  25.0
12.11 43149     38.0     33.0  5.0  25.0
10.81 41089     35.0     32.0  3.0   9.0
 9.35 14257     31.0     14.0 17.0 289.0
...
 6.54 43799 12.0 34.0 -22.0 484.0
 4.23  6235  1.0  2.0  -1.0   1.0
 5.22 11524  3.0  9.0  -6.0  36.0
18.00 17309 40.0 17.0  23.0 529.0
11.03 22225 36.0 26.0  10.0 100.0

Сумма квадратов разностей рангов (Sum D^2): 7170.00
--------------------
=== 4. Расчет коэффициента Спирмена (r_s) ===
Формула: r_s = 1 - (6 * Sum(D^2)) / (n * (n^2 - 1))
Расчет: r_s = 1 - (6 * 7170.00) / (40 * (40^2 - 1)) = 0.3274

Коэффициент корреляции Спирмена (r_s): 0.3274
--------------------
=== 

In [5]:
import pandas as pd
import numpy as np  # Добавлен импорт numpy, так как используется np.array
from scipy.stats import norm

# --- 1. Исходные данные и параметры ---
# Переменная X (Колонка 3)
X = np.array([9.26, 9.38, 12.11, 10.81, 9.35, 9.87, 8.17, 9.12, 5.88, 6.3,
        6.22, 5.49, 6.5, 6.61, 4.32, 7.37, 7.02, 8.25, 8.15, 8.72,
        7.65, 8.77, 7.0, 11.06, 9.02, 13.28, 9.27, 6.7, 6.69, 9.42,
        7.24, 5.39, 5.61, 5.59, 6.57, 6.54, 4.23, 5.22, 18.0, 11.03])

Y = np.array([47750, 50391, 43149, 41089, 14257, 22061, 52509, 14903, 11152, 16821,
        19459, 12973, 50907, 6920, 5736, 26705, 20068, 11487, 32029, 18946,
        19074, 18452, 12096, 7888, 7566, 94697, 29626, 11688, 21955, 12243,
        48141, 20122, 7612, 27404, 39648, 43799, 6235, 11524, 17309, 22225])


N = len(X)
alpha = 0.05

print(f"=== 1. Исходные данные и параметры ===")
print(f"Объем выборки (n): {N}")
print(f"Уровень значимости (alpha): {alpha}\n")


# --- 2. Ранжирование данных ---
# Функция для ранжирования (с учетом связанных рангов)
def rank_data(data):
    s = pd.Series(data)
    # method='average' для учета связанных рангов
    ranks = s.rank(method='average', ascending=True)
    return ranks.to_numpy()


R_X = rank_data(X)
R_Y = rank_data(Y)

# Объединяем в DataFrame для удобства сортировки по R_X
df = pd.DataFrame({'X': X, 'Y': Y, 'R_X': R_X, 'R_Y': R_Y})
# Сортируем по рангу X (восходящий порядок)
df_sorted = df.sort_values(by='R_X').reset_index(drop=True)

# Получаем ранги Y, соответствующие отсортированным рангам X (R_X от 1 до N)
R_Y_sorted = df_sorted['R_Y'].values

print("-" * 20)
print(f"=== 2. Подготовка рангов для подсчета P и Q ===")
print(f"Ранги Y, отсортированные по рангам X (R_X):")
print(R_Y_sorted)

# --- 3. Расчет P (Согласованные пары) и Q (Несогласованные пары) ---

P = 0  # Количество согласованных пар
Q = 0  # Количество несогласованных пар

print("\n" + "-" * 20)
print(f"=== 3. Подсчет согласованных (P) и несогласованных (Q) пар ===")

# Подсчет P и Q (перебираем все пары)
# Идем по отсортированному ряду R_Y_sorted
for i in range(N):
    # Текущий ранг R_Y_i
    rank_i = R_Y_sorted[i]

    # Считаем, сколько последующих рангов (R_Y_j, где j > i)
    # больше (P) или меньше (Q) текущего ранга R_Y_i.

    # Сумма количества рангов, больших чем rank_i, среди последующих рангов
    p_i = np.sum(R_Y_sorted[i + 1:] > rank_i)

    # Сумма количества рангов, меньших чем rank_i, среди последующих рангов
    q_i = np.sum(R_Y_sorted[i + 1:] < rank_i)

    P += p_i
    Q += q_i

    print(f"Для R_Y[{i + 1}] = {rank_i:.1f}: P+ = {p_i}, Q- = {q_i}")

# Расчет S
S = P - Q

print("\nСумма P (согласованные):", P)
print("Сумма Q (несогласованные):", Q)
print(f"Разность S = P - Q: S = {P} - {Q} = {S}")

# --- 4. Расчет коэффициента Кендалла (tau) ---
# Формула (без поправки на связанные ранги)
tau = (2 * S) / (N * (N - 1))

print("-" * 20)
print(f"=== 4. Расчет коэффициента Кендалла (r_K) ===")
print(f"Формула: r_K = (2 * S) / (n * (n - 1))")
print(f"Расчет: r_K = (2 * {S}) / (40 * (40 - 1)) = {tau:.4f}")
print(f"\nКоэффициент Кендалла (r_K): {tau:.4f}")

# Проверка значимости для N > 10
u_critical = norm.ppf(1 - alpha / 2)

# Расчет стандартной ошибки (denominator)
# sigma_S^2 = n(n-1)(2n+5) / 18
sigma_S_sq = (N * (N - 1) * (2 * N + 5)) / 18
SE_S = np.sqrt(sigma_S_sq)

# Расчет u-статистики
u_stat = S / SE_S

# Расчет критического значения для тау (u-критерия)
tau_critical_example = u_critical * np.sqrt((2 * N + 5) / (9 * N * (N - 1)))

print("-" * 20)
print(f"=== 5. Проверка значимости (u-критерий) ===")
print(f"Критическое значение u_kr для alpha=0.05: u_0.975 = {u_critical:.2f}")



print(f"\nРассчитанная u-статистика:")
print(f"Стандартная ошибка S (SE_S): sqrt((n(n-1)(2n+5))/18) = {SE_S:.2f}")
print(f"u_st = S / SE_S = {S} / {SE_S:.2f} = {u_stat:.4f}")

# --- 6. Сравнение и вывод ---
print("-" * 20)
print(f"=== 6. Вывод ===")

# Сравнение по u-статистике
print("Сравнение u-статистик:")
print(f"Рассчитанная u_st (|u_st|): {abs(u_stat):.4f}")
print(f"Критическое значение u_kr: {u_critical:.2f}")

if abs(u_stat) > u_critical:
    print(f"Результат (по u): Так как |u_st| ({abs(u_stat):.4f}) > u_kr ({u_critical:.2f}), нулевая гипотеза ОТВЕРГАЕТСЯ.")
else:
    print(f"Результат (по u): Так как |u_st| ({abs(u_stat):.4f}) <= u_kr ({u_critical:.2f}), нулевая гипотеза НЕ ОТВЕРГАЕТСЯ.")
print("-" * 20)

=== 1. Исходные данные и параметры ===
Объем выборки (n): 40
Уровень значимости (alpha): 0.05

--------------------
=== 2. Подготовка рангов для подсчета P и Q ===
Ранги Y, отсортированные по рангам X (R_X):
[ 2.  1.  9. 23. 13. 28.  5.  7. 21. 16. 38. 34. 31.  3. 24. 10. 11. 22.
 36. 27. 20. 30. 39.  8. 19. 18.  4. 15. 35. 29. 14. 37. 12. 25. 32. 26.
  6. 33. 40. 17.]

--------------------
=== 3. Подсчет согласованных (P) и несогласованных (Q) пар ===
Для R_Y[1] = 2.0: P+ = 38, Q- = 1
Для R_Y[2] = 1.0: P+ = 38, Q- = 0
Для R_Y[3] = 9.0: P+ = 31, Q- = 6
Для R_Y[4] = 23.0: P+ = 17, Q- = 19
Для R_Y[5] = 13.0: P+ = 26, Q- = 9
Для R_Y[6] = 28.0: P+ = 12, Q- = 22
Для R_Y[7] = 5.0: P+ = 31, Q- = 2
Для R_Y[8] = 7.0: P+ = 29, Q- = 3
Для R_Y[9] = 21.0: P+ = 17, Q- = 14
Для R_Y[10] = 16.0: P+ = 21, Q- = 9
Для R_Y[11] = 38.0: P+ = 2, Q- = 27
Для R_Y[12] = 34.0: P+ = 5, Q- = 23
Для R_Y[13] = 31.0: P+ = 7, Q- = 20
Для R_Y[14] = 3.0: P+ = 26, Q- = 0
Для R_Y[15] = 24.0: P+ = 12, Q- = 13
Для R_Y[16] = 

In [6]:
import pandas as pd
from scipy.stats import chi2

# --- 1. Исходные данные и параметры ---
# Данные из IMG_20251204_004050_069.jpg
# Эксперты: 2, 3, 7
data = {
    'Expert 1': [5, 5, 5, 3, 5, 4, 4, 5, 5, 4, 5, 5, 4],
    'Expert 3': [5, 5, 5, 5, 4, 5, 4, 5, 3, 5, 4, 4, 5],
    'Expert 7': [5, 5, 3, 5, 4, 5, 4, 5, 4, 2, 4, 5, 5]
}
competencies = [
    'Result', 'Organization', 'Influence', 'Communication',
    'Confidence', 'Conflict', 'Adaptability', 'Teamwork',
    'Client', 'Listening', 'Openness', 'Risks', 'Flexibility'
]
df = pd.DataFrame(data, index=competencies)

# Параметры
m = len(df.columns)  # Количество экспертов (переменных)
n = len(df.index)    # Количество компетенций (объектов)
alpha = 0.05         # Уровень значимости

print(f"=== 1. Исходные данные и параметры ===")
print(f"Количество экспертов (m): {m}")
print(f"Количество компетенций (n): {n}")
print(f"Уровень значимости (alpha): {alpha}\n")

# --- 2. Ранжирование компетенций (объектов) по каждому эксперту ---

# Ранжируем каждую колонку (эксперта) отдельно.
# Для ранжирования используется метод 'average' для учета одинаковых оценок.
df_ranks = df.apply(lambda col: col.rank(method='average', ascending=False), axis=0)
# ascending=False: более высокая оценка (5) получает меньший (лучший) ранг (1, 2, ...).

print("-" * 20)
print(f"=== 2. Ранжирование компетенций по экспертам (Rij) ===")
# Используем to_string вместо to_markdown
print(df_ranks.to_string())


# --- 3. Расчет суммы рангов (Sum_rij) и промежуточных значений ---

# 1. Сумма рангов по каждой компетенции (строке)
df_ranks['Sum_rij'] = df_ranks.sum(axis=1)
R_sum = df_ranks['Sum_rij'].values

# 2. Вычисление среднего значения суммы рангов (m(n+1)/2)
# Это константа, которая вычитается из суммы рангов
mean_sum_ranks = m * (n + 1) / 2

# 3. Разность между суммой рангов и средним значением
R_diff = R_sum - mean_sum_ranks

# 4. Квадрат разности
R_diff_sq = R_diff ** 2

# 5. Сумма квадратов разностей
S = R_diff_sq.sum()

print("-" * 20)
print(f"=== 3. Расчет промежуточных значений ===")

# Создание DataFrame для промежуточного вывода
df_intermediate = pd.DataFrame({
    'Sum_rij': R_sum,
    f'Diff (Sum - {mean_sum_ranks:.1f})': R_diff,
    f'Diff^2': R_diff_sq
}, index=competencies)

print(df_intermediate.to_string())

print(f"\nСреднее значение суммы рангов m(n+1)/2: {mean_sum_ranks:.2f}")
print(f"Сумма квадратов разностей S = Sum(Sum_rij - Mean)^2: {S:.2f}\n")


# --- 4. Расчет коэффициента конкордации Кендалла (W) ---

# Формула (2.8):
# W(m) = 12 * S / (m^2 * (n^3 - n))
W_hat = (12 * S) / (m**2 * (n**3 - n))

print("-" * 20)
print(f"=== 4. Расчет коэффициента конкордации (W) ===")
print(f"Формула: W = (12 * S) / (m^2 * (n^3 - n))")
print(f"Расчет: W = (12 * {S:.2f}) / ({m}^2 * ({n}^3 - {n})) = {W_hat:.4f}")


# --- 5. Проверка значимости (Хи-квадрат Chi^2) ---

# Для n > 7 используется критерий Хи-квадрат
# Хи-квадрат (Chi^2) = m * (n - 1) * W(m)

chi_sq_stat = m * (n - 1) * W_hat

# Определение критического значения Chi^2
# Степени свободы: df = n - 1
df_chi = n - 1
chi_sq_critical = chi2.ppf(1 - alpha, df_chi)

print("-" * 20)
print(f"=== 5. Проверка значимости (Chi^2-критерий) ===")

print(f"Рассчитанное значение Chi^2_st: m * (n-1) * W = {m} * ({n}-1) * {W_hat:.4f} = {chi_sq_stat:.4f}")

print(f"\nКритическое значение Chi^2_kr (alpha={alpha}; df={df_chi}):")
print(f"Chi^2_kr = {chi_sq_critical:.4f}")

# --- 6. Вывод ---

print("-" * 20)
print(f"=== 6. Вывод ===")

# Условие: Chi^2_st > Chi^2_kr
if chi_sq_stat > chi_sq_critical:
    print(f"Результат: Так как Chi^2_st ({chi_sq_stat:.4f}) > Chi^2_kr ({chi_sq_critical:.4f}), нулевая гипотеза об отсутствии согласованности ОТВЕРГАЕТСЯ.")
    print(f"Это означает, что коэффициент конкордации значимо отличается от нуля с вероятностью 95%.")
else:
    print(f"Результат: Так как Chi^2_st ({chi_sq_stat:.4f}) <= Chi^2_kr ({chi_sq_critical:.4f}), нулевая гипотеза об отсутствии согласованности НЕ ОТВЕРГАЕТСЯ.")
    print(f"Следовательно, нет достаточных оснований утверждать, что оценки экспертов значимо согласованы на уровне alpha=0.05.")

=== 1. Исходные данные и параметры ===
Количество экспертов (m): 3
Количество компетенций (n): 13
Уровень значимости (alpha): 0.05

--------------------
=== 2. Ранжирование компетенций по экспертам (Rij) ===
               Expert 1  Expert 3  Expert 7
Result              4.5       4.5       4.0
Organization        4.5       4.5       4.0
Influence           4.5       4.5      12.0
Communication      13.0       4.5       4.0
Confidence          4.5      10.5       9.5
Conflict           10.5       4.5       4.0
Adaptability       10.5      10.5       9.5
Teamwork            4.5       4.5       4.0
Client              4.5      13.0       9.5
Listening          10.5       4.5      13.0
Openness            4.5      10.5       9.5
Risks               4.5      10.5       4.0
Flexibility        10.5       4.5       4.0
--------------------
=== 3. Расчет промежуточных значений ===
               Sum_rij  Diff (Sum - 21.0)  Diff^2
Result            13.0               -8.0   64.00
Organization  